### Install required packages

In [ ]:
%pip install python-dotenv
%pip install requests
%pip install azure-ai-inference
%pip install --upgrade python-certifi-win32
%pip install --upgrade certifi
%pip install pip-system-certs

### Load configurations

In [ ]:
from dotenv import dotenv_values,load_dotenv
from azure.ai.inference import ChatCompletionsClient, EmbeddingsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference.models import SystemMessage, UserMessage
import sys
sys.path.append('../common')
from helper import Helper
import os

env = os.getenv('GITHUB_ENV') or 'local'
env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-QA'

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")
helper = Helper(environment, "genai/azure/deployments", api_key, region_override)

helper.log('GITHUB_ENV: ' + env)
helper.log('ENV_NAME: ' + env_name)

### Get Deployment ID List

In [ ]:
env_config = helper.read_json_config(env_name)

# Check if deployment_info is found
assert env_config is not None, f"Deployment information not found for environment: {env_name}"

chat_model_deployments_list = env_config["api_configuration"]["inference"]['chat_models']

flattened_chat_model_deployments_list = [
    {"api_version": deployment["api_version"], "model": model}
    for deployment in chat_model_deployments_list
    for model in deployment["models"]
]

helper.log("Chat Models:")
helper.log(chat_model_deployments_list)
helper.log(flattened_chat_model_deployments_list)

### Chat Completion - Invalid Subscription key

In [ ]:
helper.log("Test : Validate the invalid api key in chat completion requets")

for deployment_id in flattened_chat_model_deployments_list:
    # Create a client
    try:
        helper.log(deployment_id['model'])
        client = ChatCompletionsClient(
        endpoint=f"https://{environment}/genai/azure/inference",
        credential=AzureKeyCredential(""),
        headers = {
            "Ocp-Apim-Subscription-Key": '%',
            'x-kpmg-charge-code': '803000015283',
            "azureml-model-deployment": deployment_id['model'],
            "x-kpmg-region-override": region_override
        }
    )

    # Create a completion request
        response = client.complete(
            messages=[
                SystemMessage(content="You are a helpful assistant."),
                UserMessage(content="What is the capital of France?"),
            ]
        )

    except Exception as err:
        error = err
        #helper.log(deployment_id['model'])
        helper.log('Error Message: ', error.message[135:] + '\n')
        helper.log('Status Code: ', str(error.status_code))
        helper.log('')

        # Validate the response
        assert error.status_code == 401
        assert "Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription." in error.message
        continue

helper.log('------------------End of test------------------')



### Chat Completion - Invalid model name

In [ ]:
helper.log("Test : Validate the invalid model name in chat completion requets")

# Create a client
client = ChatCompletionsClient(
    endpoint=f"https://{environment}/genai/azure/inference",
    credential=AzureKeyCredential(""),
    headers={
        "Ocp-Apim-Subscription-Key": api_key,
        "azureml-model-deployment": 'IamInvalidModel',
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override
    },
)

try:
    # Create a completion request
    _ = client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant."),
            UserMessage(
                content="Tell me a bit about your your creator, and the model that you run on?"
            ),
        ]
    )
except Exception as err:
    error = err
    helper.log(error)

# Validate the response
assert error.status_code == 404
assert "No deployments matched the requested ID, or you do not have access to it" in error.message

helper.log('------------------End of test------------------')

### Chat Completion - Empty Request Body

In [ ]:
helper.log("Test : Validate the empty request body in chat completion requets")

for deployment_id in flattened_chat_model_deployments_list:
    # Create a client
    try:
        client = ChatCompletionsClient(
        endpoint=f"https://{environment}/genai/azure/inference",
        credential=AzureKeyCredential(""),
        headers = {
            "Ocp-Apim-Subscription-Key": api_key,
            'x-kpmg-charge-code': '803000015283',
            "azureml-model-deployment": deployment_id,
            "x-kpmg-region-override": region_override
        }
    )

    # Create a completion request
        response = client.complete(

        )

    except Exception as err:
        error = err
        try:
            helper.log(f"Model: {deployment_id['model']}")
        except UnicodeEncodeError:
            helper.log("Model: [Unicode model name]")
        helper.log(error)
        assert "missing required argument: messages" in str(error)
        helper.log('')
    
    except TypeError:
        continue

helper.log('------------------End of test------------------')

### Chat Completion - Invalid Request Body

In [ ]:
helper.log("Test : Validate the invalid request body in chat completion requets")

for deployment_id in flattened_chat_model_deployments_list:
    # Create a client
    try:
        client = ChatCompletionsClient(
        endpoint=f"https://{environment}/genai/azure/inference",
        credential=AzureKeyCredential(""),
        headers = {
            "Ocp-Apim-Subscription-Key": api_key,
            'x-kpmg-charge-code': '803000015283',
            "azureml-model-deployment": deployment_id['model'],
            "x-kpmg-region-override": region_override
        }
    )

    # Create a completion request
        response = client.complete(
            messages=[
                SystemMessage(content=123),
                UserMessage(content=False),
            ]
        )

    except Exception as err:
        error = err
        try:
            helper.log(f"Model: {deployment_id['model']}")
        except UnicodeEncodeError:
            helper.log("Model: [Unicode model name]")
        try:
            helper.log(f"Error message: {error.message}, Status Code: {error.status_code}")
        except UnicodeEncodeError:
            helper.log(f"Error message: [Unicode error], Status Code: {error.status_code}")
        helper.log('')

        # Validate the response
        assert error.status_code == 400
        assert "Bad Request" in error.message
        continue


helper.log('------------------End of test------------------')

### "Info" and "Get All Routes" Invalid Subscription key

In [ ]:
helper = Helper(environment, "genai/azure/inference", 'ghh')

In [ ]:
helper.log("Test : Validate the invalid api key in Info API")

# Info API end point
for deployment_id in flattened_chat_model_deployments_list:
    try:
        headers = {
            "Ocp-Apim-Subscription-Key": 'fg',
            'x-kpmg-charge-code': '803000015283',
            "azureml-model-deployment": deployment_id['model']
        }
        try:
            helper.log(f"Model: {deployment_id['model']}")
        except UnicodeEncodeError:
            helper.log("Model: [Unicode model name]")
        response = helper.get("/info")
        assert response.status_code == 401

    except:
        helper.log('Status is not 401')
        
helper.log('------------------End of test------------------')

In [ ]:
helper.log("Test : Validate the invalid api key in Get All Routes API")

# "Get All Routes" API end point
for deployment_id in flattened_chat_model_deployments_list:
    try:
        headers = {
            "Ocp-Apim-Subscription-Key": 'fg',
            'x-kpmg-charge-code': '803000015283',
            "azureml-model-deployment": deployment_id['model']
        }
        try:
            helper.log(f"Model: {deployment_id['model']}")
        except UnicodeEncodeError:
            helper.log("Model: [Unicode model name]")
        response = helper.get("/listRoutes")
        assert response.status_code == 401

    except:
        helper.log('Status is not 401')

helper.log('------------------End of test------------------')


